### classify stackoverflow posts by considering each document a simple mean of its individual word embeddings (but fine-tuning them), using one-vs-all SVM

In [1]:
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

Using TensorFlow backend.


In [2]:
from keras import backend as K

In [3]:
SEED=np.random.randint(1,1000)
SEED

416

In [4]:
np.random.seed(seed=SEED)

In [5]:
texts, labels = files_helper.read_stackoverflow_sample_small()

In [6]:
preprocessed_texts = texts

In [22]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=3

In [8]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(preprocessed_texts)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [9]:
[inverse_word_index[idx] for idx in sequences[0]]

['apply',
 'onclick',
 'event',
 'to',
 'an',
 'option',
 'i',
 'am',
 'using',
 'zend',
 'form',
 'to',
 'create',
 'a',
 'form',
 'i',
 'am',
 'also',
 'using',
 'for',
 'javascript',
 'this',
 'gt',
 'radio',
 'alone',
 'array',
 'label',
 'gt',
 'are',
 'you',
 'going',
 'to',
 'be',
 'taking',
 'part',
 'with',
 'anyone',
 'else',
 'required',
 'gt',
 'true',
 'onclick',
 'gt',
 'gt',
 'array',
 'yes',
 'gt',
 'yes',
 'no',
 'gt',
 'no',
 'at',
 'the',
 'moment',
 'the',
 'onclick',
 'event',
 'works',
 'if',
 'any',
 'option',
 'is',
 'selected',
 'how',
 'do',
 'i',
 'get',
 'it',
 'to',
 'work',
 'for',
 'just',
 'yes',
 'being',
 'selected']

In [10]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [11]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [12]:
# tag position => fraction of docs having that tag
tag_probabilities_index = tags_helper.get_probabilities_index(binary_labels)

In [13]:
# word => embedding
embeddings_index = files_helper.read_glove(d=EMBEDDING_DIM)

In [14]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

data.shape,labels_1.shape

((32000, 1000), (32000, 1209))

In [15]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [23]:
%%time

num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = True))

model.add(GlobalAvgPool1D())
model.add(Dense(num_labels))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam')

model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    verbose=1,
                     validation_data=(X_val,Y_val))

Train on 25600 samples, validate on 6400 samples
Epoch 1/3
25600/25600 [==============================] - 40s - loss: 0.0788 - val_loss: 0.0119
Epoch 2/3
25600/25600 [==============================] - 40s - loss: 0.0115 - val_loss: 0.0114
Epoch 3/3
25600/25600 [==============================] - 40s - loss: 0.0114 - val_loss: 0.0114
CPU times: user 1min 43s, sys: 19.1 s, total: 2min 2s
Wall time: 2min 2s


In [24]:
get_pooling_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[1].output])

In [25]:
new_X_train = []
for raw_doc_features in X_train:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1)])
    new_X_train.append(layer_output[0].ravel())
new_X_train = np.vstack(new_X_train)

new_X_train.shape

(25600, 100)

In [26]:
new_X_val = []
for raw_doc_features in X_val:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1)])
    new_X_val.append(layer_output[0].ravel())
new_X_val = np.vstack([new_X_val])

new_X_val.shape

(6400, 100)

In [31]:
clf = OneVsRestClassifier(Pipeline([
    ('clf', svm.LinearSVC()),
]),n_jobs=-1)
parameters = [
    { 
          "estimator__clf__penalty": ["l1"],
          "estimator__clf__dual":[False],
          "estimator__clf__multi_class":["ovr"],
          "estimator__clf__tol": [0.001],

    },
]

In [ ]:
best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(new_X_train,Y_train)
    
    Y_pred_train = clf.predict(new_X_train)    
    Y_pred_val = clf.predict(new_X_val)
    
    train_score = f1_score(Y_train,Y_pred_train,average='micro')
    val_score = f1_score(Y_val,Y_pred_val,average='micro')
    
    current_score = val_score
    
    print("train micro-F1: {}".format(train_score))
    print("val micro-F1: {}".format(val_score))
    print("grid: {}".format(g))
    print("")
    
    if current_score > best_score:
        best_score = current_score
        best_grid = g

train micro-F1: 0.02955222390468742
val micro-F1: 0.026679054950889302
grid: {'estimator__clf__dual': False, 'estimator__clf__penalty': 'l1', 'estimator__clf__tol': 0.001, 'estimator__clf__multi_class': 'ovr'}



Process ForkPoolWorker-40:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/externals/joblib/pool.py", line 362, in get
    return recv()
